In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.stats import chi2, norm, cauchy
from iminuit import Minuit
from iminuit.cost import LeastSquares
# Cambiare directory per spostarsi nella cartella principale
import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')

import sys
sys.path.append('Config')
from lablib import *

# Definizioni

# Vecchi dati

## 53.5 V

In [ ]:
#leggo il file dalla riga 66 in poi, escludendo il testo e il nome delle colonne
df = pd.read_csv('Dati/Parte 2/Riprendere/Spettro CsI/Na_53.5_th-20.txt', delim_whitespace = 'true',  skiprows = 65, encoding='ISO-8859-1', header = None)
df.columns = ['ADC', 'counts'] #ho assegnato il nome alle colonne del dataframe
plt.plot(df['ADC'], df['counts'])

In [ ]:
#seleziono dati del primo picco
# Definisco gli indici delle righe copiate
start_index =80  # Indice di inizio (inclusivo)
end_index = 105   # wIndice di fine (esclusivo)
# Copio le righe selezionate in un nuovo dataframe
df_filtrato = df.iloc[start_index:end_index]
x_primo_picco = df_filtrato['ADC']
y_primo_picco = df_filtrato['counts']
plt.plot(x_primo_picco, y_primo_picco)
errori_counts = np.array(np.sqrt(df_filtrato['counts']))
plt.errorbar(x_primo_picco, y_primo_picco, yerr=errori_counts, label='Data')
#print (errori_counts)
errori_counts1 = np.array(np.sqrt(df['counts']))

In [ ]:
LS_fit(x_primo_picco, y_primo_picco, errori_counts, gauss, mu = 6153, sigma = 611, S = 1391)

In [ ]:
LS_fit(df['ADC'], df['counts'], errori_counts1, gauss,mu = 6157, sigma = 640, S = 1350)



## 54.5 V

In [ ]:
#importo prima il file di dati + fondo, poi quello solo fondo e faccio la sottrazione dei due plot
df54 = pd.read_csv('Dati/Parte 2/Riprendere/Spettro CsI/Na_54.5_th-20.txt', delim_whitespace= 'True', skiprows = 1, encoding='ISO-8859-1', header = None)
df54.columns = ['ADC', 'counts'] #ho assegnato il nome alle colonne del dataframe
err_counts_54 = np.array(np.sqrt(df54['counts']))
df54_bkg = pd.read_csv('Dati/Parte 2/Riprendere/Spettro CsI/Bg_54.5_th-20.txt', delim_whitespace= 'True', skiprows = 65, encoding='ISO-8859-1', header = None)
df54_bkg.columns = ['ADC', 'counts'] #ho assegnato il nome alle colonne del dataframe
err_counts_54_bkg = np.array(np.sqrt(df54_bkg['counts'][:500]))
fig, ax = plt.subplots(figsize = (30,6))
plt.plot(df54['ADC'], df54['counts'])
plt.errorbar(df54['ADC'], df54['counts'], yerr=err_counts_54)
plt.plot(df54_bkg['ADC'][:500], df54_bkg['counts'][:500])
plt.errorbar(df54_bkg['ADC'][:500], df54_bkg['counts'][:500], yerr=err_counts_54_bkg)
plt.grid()


In [ ]:
#taglio il vettore per il fit primo picco
start_index = 173
end_index = 210
df_filtrato_54_primo_picco = df54.iloc[start_index:end_index]
err_54_primo_picco = np.array(np.sqrt(df_filtrato_54_primo_picco['counts']))

In [ ]:
pars, vals, errs, *others = LS_fit(df_filtrato_54_primo_picco['ADC'], df_filtrato_54_primo_picco['counts'], err_54_primo_picco, gauss, mu = 14000, sigma = 1000, S = 700)

In [ ]:
for i in range(len(pars)):
    print(f'{pars[i]} = {vals[i]} +/- {errs[i]}')

In [ ]:
start_index = 337
end_index = 391
df_filtrato_54_secondo_picco = df54.iloc[start_index:end_index]
err_54_secondo_picco = np.array(np.sqrt(df_filtrato_54_secondo_picco['counts']))

In [ ]:
pars, vals, ers, *other = LS_fit(df_filtrato_54_secondo_picco['ADC'], df_filtrato_54_secondo_picco['counts'], err_54_secondo_picco, gauss, mu = 31000, sigma = 1400, S = 10)

In [ ]:
#provo a sottrarre i due daataframe e vedo cosa ne esce
y_54_no_bkg = []
err_y_54 = []
for i in range (500):
    y_54_no_bkg.append( df54['counts'][i] - df54_bkg['counts'][i])
    err_y_54.append(math.sqrt(err_counts_54[i]**2 + err_counts_54_bkg[i]**2))
fig, ax = plt.subplots(figsize = (30,6))
plt.plot(df54['ADC'], y_54_no_bkg)
plt.errorbar(df54['ADC'], y_54_no_bkg, yerr=err_y_54, label='Data')


# Nuovi dati

## Sodio

In [ ]:
df55 = pd.read_csv('Dati/Parte 2/CsI/Sodio_55.5.txt', delim_whitespace= 'True', skiprows = 65, encoding='ISO-8859-1', header = None)
df55.columns = ['ADC', 'counts'] #ho assegnato il nome alle colonne del dataframe
err_counts_55 = np.array(np.sqrt(df55['counts']))
df55_bkg = pd.read_csv('Dati/Parte 2/CsI/Bkg_54.5V.txt', delim_whitespace= 'True', skiprows = 65, encoding='ISO-8859-1', header = None)   
df55_bkg.columns = ['ADC', 'counts'] #ho assegnato il nome alle colonne del dataframe
# Allunga df55_bkg per avere la stessa lunghezza di df55
diff = len(df55) - len(df55_bkg)
if diff > 0:
    last_adc = df55_bkg['ADC'].iloc[-1]
    adc_step = df55_bkg['ADC'].diff().mode()[0] if len(df55_bkg['ADC']) > 1 else 1
    new_adc = [last_adc + adc_step * (i + 1) for i in range(diff)]
    df55_bkg = pd.concat([
        df55_bkg,
        pd.DataFrame({'ADC': new_adc, 'counts': [0]*diff})
    ], ignore_index=True)
err_counts_55_bkg = np.array(np.sqrt(df55_bkg['counts']))
fig, ax = plt.subplots(figsize = (15,6))
plt.plot(df55['ADC'], df55['counts'], linewidth = 2, label = 'spettro con sorgente di sodio')
plt.xlabel('ADC channels')
plt.ylabel('Counts')
# plt.errorbar(df55['ADC'], df55['counts'], yerr=err_counts_55)
plt.scatter(df55_bkg['ADC'], df55_bkg['counts'], s = 3.5, label = 'Background', color = 'red')
plt.legend(fontsize = 23)
plt.grid()
ax.set_xlabel('ADC channels', fontsize = 23)
ax.set_ylabel('Counts', fontsize = 23)
# plt.errorbar(df55_bkg['ADC'], df55_bkg['counts'], yerr=err_counts_55_bkg)

In [ ]:
start_index = 250
end_index = 340
df_filtrato = df55.iloc[start_index:end_index]
err_df55_primo_picco = np.array(np.sqrt(df_filtrato['counts']))

In [ ]:
pars, vals, errs, *others = LS_fit(df_filtrato['ADC'], df_filtrato['counts'], err_df55_primo_picco, gauss, mu = 14000, sigma = 1000, S = 700)

In [ ]:
for i in range(len(pars)):
    print(f'{pars[i]} = {vals[i]} +/- {errs[i]}')

In [ ]:
risol_na = 2.355 * vals[1] / vals[0]

## Cobalto

In [ ]:
Co = read_corretto(path = 'Dati/Parte 2/CsI/Cobalto_55.5V.txt', titles= ['ADC', 'Counts'])
Co_bkg = read_corretto(path = 'Dati/Parte 2/CsI/Bkg_54.5V.txt', titles= ['ADC', 'Counts'])
Fig, axes = plt.subplots( figsize = (15,6))
axes.plot(Co['ADC'], Co['Counts'], linewidth = 0.8, label = 'spettro con sorgente di cobalto')
axes.set_xlabel('ADC channels')
axes.set_ylabel('Counts')
axes.scatter(df55_bkg['ADC'][39:240], df55_bkg['counts'][39:240], s = 0.5, label = 'Background', color = 'red')
axes.legend()
axes.grid()
# axes[0].scatter(Co_bkg['ADC'], Co_bkg['Counts'], color = 'red', s = 0.5)

In [ ]:
start_index = 110 - 65
end_index = 160 - 65
pars, vals, errs, *others = LS_fit(Co['ADC'][start_index:end_index], Co['Counts'][start_index:end_index], np.sqrt(Co['Counts'][start_index:end_index]), gauss_pol3, mu = 5000, sigma = 1000, S = 500, b = 0, c = 0, d = 0, a = 0)

In [ ]:
risol_Co = 2.355 *vals[1]/vals[0]

## Risoluzione

In [ ]:
print(f'Risoluzione Cobalto 122 keV: {risol_Co:.3e}','\n',
       f'Risoluzione Sodio 511 keV: {risol_na:.3e}')